In [3]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv")
setorder(Centroids, site)
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")
setnames(PropSamp, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#read in the ROMS simulation connectivity table with metadata, not yet subsetted (*but check this)
#SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableWithMetaLongForm.csv")

#add in the numbers of particles seeded at each site
SeededParticles <- fread("~/oceanography/ROMS/data/Particles_Per_Release_Site_Renamed.csv")
setnames(SeededParticles,c("source", "daily_particles_released")) 
#DateJoin <- SeededParticles[DateJoin, on="source"][, particles_released_daily := as.numeric(particles_released_daily)] 

#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
unsampled_sites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
sand_flats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 
unrealistic_sources <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan") 
#make the constant inputs for the kernel fitting function
#distance matrix using the centroids with combined Magbangon
### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
setorder(Area, site)
reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids, by="site")[, "site"][, index := .I] #add the row number as the unique site index

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][,#change NA's to 0
    -"time_frame"]#drop the time_frame column, we can key with end_year
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][
    is.na(n_offs_gen), n_offs_gen := 0][,#change NA's to 0
    -"time_frame"]#drop the time_frame

SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this



In [4]:
#make a table of the dates of release for simulations, for calculating the number of particles released in each time frame
season1 <- data.table(date=seq(as.Date("2010-10-01"), as.Date("2011-05-31"), by="days"))

season2 <- data.table(date=seq(as.Date("2011-10-01"), as.Date("2012-05-31"), by="days"))

season3 <- data.table(date=seq(as.Date("2012-10-01"), as.Date("2013-05-31"), by="days"))

season4 <- data.table(date=seq(as.Date("2013-10-01"), as.Date("2014-04-18"), by="days"))

AllDates <- rbind(season1, season2, season3, season4)

#mark the monsoon seasons, based on the same criteria I used for the parentage indirectly through the growth estimates
NEM <- c(11, 12, 1, 2, 3, 4, 5, 6)
SWM <- c(7, 8, 9, 10)

AllDates[,date := ymd(date)][, #format as ymd
             sim_monsoon := ifelse(month(date) %in% NEM, "NEM", "SWM")][,#mark monsoon season based on month
             sim_year:=year(date)][,#add year column
            year_sampled:= ifelse(date %in% season1$date, 2011, ifelse(date %in% season2$date, 2012, ifelse(date %in% season3$date, 2013, 2014)))]#and then add a year_sampled for the empircal sampling season of that particle

ReleaseDays <- AllDates[, .(num_release_days_seasonal=.N), by=c("year_sampled", "sim_monsoon")][, num_release_days_annual:= sum(num_release_days_seasonal), by=year_sampled]

total_release_days <- AllDates[year_sampled %in% c(2012, 2013, 2014), .N]#for the all year kernel- how many days of the simulation conincide with our particle sampling?
total_release_days #should be 687


[1] 687

In [40]:
##prep biophysical connectivity matrix
##outside of the loop, trim this to only be the destinations we sampled
#SourceJoin <- SurveyData[SimConn, on = .(site = source, year=year_sampled)]
#setnames(SourceJoin, skip_absent=TRUE, c("site", "n_offs_gen", "prop_anem_samp", "total_anems", "num_females"), c("source", "source_num_rec_sampled_annual",  "source_prop_anem_samp", "source_total_anems", "source_num_females"))
#DestJoin <- SurveyData[SourceJoin, on = .(site = dest, year)]
#setnames(DestJoin, skip_absent=TRUE, c("site", "n_offs_gen", "prop_anem_samp", "total_anems", "num_females"), c("dest", "dest_num_rec_sampled_annual",  "dest_prop_samp", "dest_total_anems", "dest_num_females"))

#SimConn <- DestJoin[source %!in% unrealistic_sources & dest %!in% unrealistic_sources & year %in% c(2012, 2013, 2014)][#sand flats and Pangasugan are not realistic source or destination sites because there's almost no habitat. Safe to drop, but keep the rest of the possibilities so we can subsample iteratively all possibilities.
#    , daily_particles_released := as.numeric(daily_particles_released)] #change from integer to numeric
#SimConn <- ReleaseDays[SimConn, on=.(year_sampled=year, sim_monsoon)]#join in the info for number of release days in the time frame
#SimConn <- kernels[Year %in% c("2012", "2013", "2014")][, year:=as.integer(Year)][,c("year", "NumParentageMatches")][SimConn, on=.(year=year_sampled)]#add in a column for the observed number of parentage matches
##rename the monsoon column in the full table for consistency
#setnames(SimConn, c("sim_monsoon", "NumParentageMatches"), c("monsoon", "num_route_parentage_matches")) #get rid of upper case and inconsistent naming
#
##each year will require a different set of survey data, so make a list of each and index by site for fast look up
#SampledTable <- SurveyData[prop_anem_samp >0, c("year", "site")]#previously named PropSampTable
#
##make sure all sampled sites are represented when joining the survey data to the sampled simulation- this chunk has the tables to add to a subsampled particle table. no need for the full
#SampTable <- rbind(SurveyData[prop_anem_samp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(source=site, dest=site, year=year)][, #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
#     c("year", "source", "dest")][, monsoon := "NEM"], SurveyData[prop_anem_samp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(source=site, dest=site, year=year)][, #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
#     c("year", "source", "dest")][, monsoon := "SWM"])
#UnqSurvey <- unique(SampTable, by=c("source", "dest", "year", "monsoon"))#add in the diff Monsoon seasons so there are complete parentage matrices later
#AddDest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0][, c("year", "site")][, monsoon := "NEM"], 
#                  SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0][ , c("year", "site")][, monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table
#setcolorder(SimConn, c("particle_id", "source", "dest", "year", "monsoon", "date"))
#
##at this point, we can make the raw number assignment matrix, but we want to make a normalized version that is num assigned from a source to a destination/ num released from that source
##fwrite(SimConn, file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm.csv")

ERROR: Error in eval(expr, envir, enclos): object 'DestJoin' not found


__Skip the joining of tables that takes forever and read in the__ 

In [18]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm.csv")

In [41]:
#each year will require a different set of survey data, so make a list of each and index by site for fast look up
SampledTable <- SurveyData[prop_anem_samp >0, c("year", "site")]#previously named PropSampTable

#make sure all sampled sites are represented when joining the survey data to the sampled simulation- this chunk has the tables to add to a subsampled particle table. no need for the full
SampTable <- rbind(SurveyData[prop_anem_samp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(source=site, dest=site, year=year)][, #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
     c("year", "source", "dest")][, monsoon := "NEM"], SurveyData[prop_anem_samp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(source=site, dest=site, year=year)][, #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
     c("year", "source", "dest")][, monsoon := "SWM"])
UnqSurvey <- unique(SampTable, by=c("source", "dest", "year", "monsoon"))#add in the diff Monsoon seasons so there are complete parentage matrices later
AddDest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0][, c("year", "site")][, monsoon := "NEM"], 
                  SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0][ , c("year", "site")][, monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table


In [ ]:
#First, figure out how to get normalized conn values for all time frames
#then, test kernel fitting function on full matrix      
#last, start testing likelihood function build

In [8]:
#at this point, we can make the raw number assignment matrix, but we want to make a normalized version that is num assigned from a source to a destination/ num released from that source

AllYearsRec <- SimConn[, .(total_particles_rec = .N), by= c("source", "dest")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
AllYearsRelease <- unique(SimConn[,.(total_particles_released = as.numeric(daily_particles_released)*as.numeric(total_release_days)), by= c("source")], by="source") #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs
#join recruited and released tables together and make a column for the normalized values
AllYearsNormConn <-  AllYearsRelease[AllYearsRec, on="source"][
    , source_norm_rec := total_particles_rec/total_particles_released]

#check that they sum to =< 1
AllYearsNormConn[,sum(source_norm_rec), by="source"]#nothing should be greater than 1. It isn't- great

#make sure all possible routes are represented!!*
#cast into wide format
FullBiophysMatNorm <- as.matrix(rbind(dcast(AllYearsNormConn[source != "Other", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(AllYearsNormConn[source == "Other" , .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)][, source := "unknown"], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
dim(FullBiophysMatNorm )
FullBiophysMatNorm[is.na(FullBiophysMatNorm)] <- 0 #change NAs to zeros


In [51]:
FullBiophysMatNorm

CAI,Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Gabas,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Sitio Tugas,Tamakin Dacot,Visca,Wangag
0.668285782,1.532165e-07,0.000000e+00,1.178589e-08,0.000000e+00,1.414306e-07,5.892943e-08,5.892943e-09,2.533966e-07,1.885742e-07,2.946472e-08,1.178589e-08,5.303649e-08,8.839415e-08,0.000000e+00,0.000000e+00,3.535766e-08,1.473236e-07,5.892943e-08
0.011848793,1.470278e-03,1.661032e-03,2.595729e-03,3.448256e-04,0.000000e+00,7.043246e-05,1.352890e-03,1.154799e-03,2.993379e-04,1.041813e-04,1.995586e-04,5.590576e-04,1.379302e-03,5.502536e-04,4.900925e-04,8.070386e-05,1.467343e-06,4.152580e-04
0.010873010,1.123985e-03,3.289783e-03,4.287576e-03,2.964032e-04,1.173874e-05,1.760811e-05,3.594990e-04,1.119583e-03,1.892872e-04,1.144527e-04,2.142321e-04,7.908978e-04,1.323543e-03,9.713809e-04,4.724844e-04,2.934686e-06,0.000000e+00,1.848852e-04
0.009368984,1.135723e-03,1.072628e-03,3.075551e-03,2.729258e-04,0.000000e+00,1.173874e-05,3.873785e-04,7.659530e-04,7.043246e-05,8.217120e-05,2.890665e-04,1.027140e-03,1.116648e-03,9.787177e-04,4.548763e-04,5.869371e-06,2.934686e-06,1.305935e-04
0.007389538,6.559022e-04,1.333815e-03,2.585458e-03,2.321336e-03,1.467343e-06,5.722637e-05,5.622858e-03,6.397615e-04,1.291262e-04,2.699911e-04,4.196600e-04,1.229633e-03,7.776917e-04,1.057954e-03,2.701378e-03,1.173874e-05,2.934686e-06,1.393976e-04
0.000513570,1.716791e-04,1.379302e-04,5.194394e-04,2.677901e-03,2.502993e-02,2.377095e-04,1.760811e-03,1.511363e-04,4.548763e-05,1.194417e-03,1.810701e-03,8.252336e-03,2.435789e-04,1.558465e-02,2.299326e-03,1.804832e-04,1.260447e-02,3.228154e-05
0.004512813,1.540710e-05,7.336714e-07,8.070386e-06,9.317627e-05,1.467343e-05,2.549508e-03,2.934686e-05,1.027140e-05,2.934686e-06,2.054280e-05,2.641217e-05,6.676410e-05,4.548763e-05,2.054280e-05,4.108560e-05,9.662452e-04,5.575903e-05,5.869371e-06
0.006879637,7.894304e-04,1.666901e-03,2.696242e-03,1.447534e-03,3.668357e-05,3.514286e-04,3.006585e-03,5.524546e-04,1.628751e-04,4.944945e-04,8.253803e-04,1.355091e-03,1.385172e-03,1.580328e-03,1.258246e-03,1.958903e-04,5.649270e-05,2.575187e-04
0.008655855,1.113713e-03,2.732192e-03,3.950087e-03,2.597197e-04,0.000000e+00,6.749777e-05,1.181211e-03,6.779124e-04,5.458515e-04,6.016106e-05,1.217895e-04,7.806264e-04,2.407910e-03,3.947152e-04,3.448256e-04,3.374888e-05,1.907546e-05,7.952998e-04
0.004868643,9.757830e-04,2.173135e-03,1.178276e-03,2.934686e-06,0.000000e+00,0.000000e+00,1.364629e-04,1.172407e-03,9.258933e-04,1.320609e-05,1.467343e-06,5.575903e-05,4.601587e-03,1.467343e-06,7.483448e-05,0.000000e+00,0.000000e+00,8.217120e-04


In [86]:
#make a parentage matrix for the whole biophysical results
FullBiophysMat <- as.matrix(rbind(dcast(SimConn[source != "Other" & source != "CAI" & dest!= "CAI" , .(source, dest)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    , parentage :=1][ #mark each row as a parentage match, because at this point I'm using all particles as matches for the simulations
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="parentage", fun.aggregate = sum),#[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
   # ,-"source"], #remove the source column after casting
      dcast(SimConn[source == "Other" | source == "CAI"& dest != "CAI" , .(source, dest)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          , parentage :=1][order(source, dest)][, source := "unknown"], source ~ dest, value.var="parentage", fun.aggregate = sum)))#[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
dim(FullBiophysMat)

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 has 20 columns, inconsistent with item 1 which has 19 columns. To fill missing columns use fill=TRUE.


In [ ]:
FullBiophysMat

In [88]:
 dcast(SimConn[source == "Other" | source == "CAI"& dest != "CAI" , .(source, dest)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          , parentage :=1][order(source, dest)][, source := "unknown"], source ~ dest, value.var="parentage", fun.aggregate = sum)

source,CAI,Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Gabas,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Sitio Tugas,Tamakin Dacot,Visca,Wangag
unknown,42536082,18877,21764,24086,21110,4055,53871,33600,18610,20296,7217,8742,12101,38936,19517,12731,150187,8072,18392


In [ ]:
#next test likelihood function and kernel fitting with this matrix

In [79]:
x <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1)) #put inputs into a list because that's the bbmle format
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
Sim2012_4Fit
#Next, do grid search to get the likelihood profile to compare to genetics!


Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -3, theta = 1), 
    method = "L-BFGS-B", data = x, lower = c(-10, 0.15), upper = c(10, 
        8), control = list(maxit = 500))

Coefficients:
        k     theta 
-1.823630  1.360603 

Log-likelihood: -1158730637 

In [ ]:
x

In [ ]:
dim(FullBiophysMat)

In [ ]:
dcast(DestSampled[source == "Other", .(source, dest)][, parentage :=1][order(source, dest)], source ~ dest, value.var="parentage", fun.aggregate = sum)

In [ ]:
#Mike Bode said I should be less aggressively subsampling the biophysical data, but how should I do that? The kernel fitting still requires that I have a prop samp vector... but when I tried putting 1's in that vector and using all of the particles for fitting I got crazy 

In [ ]:
#sample the particle data
SimSample <- DestSampled[, .SD[sample(.N, dest_n_rec_annual)], by = c("year", "dest")] #, prob=surv_weight #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys
check1 <- nrow(SimSample)

#assign parentage
SimParentage <- SimSample[source_prop_samp > 0][, .SD[sample(.N, num_parentage_matches)], by = .(year)][#Prob=SurvWeight, #now randomly assign parentage or not parentage, based on how well we sampled the source and the number of parentage matches we had in that year
                , parentage := 1]
#for faster searching, set keys
setkey(SimParentage, particle_id)
setkey(SimSample, particle_id)

l <- list(SimSample[particle_id %!in% SimParentage$particle_id][, parentage := 0], SimParentage)
SimSample <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = NULL)[, c("year", "source", "dest", "parentage", "monsoon")] #add back in to the unassigned particles, select only the columns necessary

#check results, for testing loop only
#nrow(SimSample)==check1 #should be TRUE
#sum(SimSample$parentage) #should be 37

In [ ]:
sum(SimSample$parentage)

In [ ]:

#calculate the unassigned row
Unassigned <- unique(SimSample[parentage==0][#not counting parentage!
    , num_sampled := .(.N), by= c("dest", "year", "monsoon")], by=c("dest", "year", "monsoon"))[, -"source"]
#add destinations not sampled in loop iteration to unassigned 
Unassigned <- Unassigned[AddDest, on=.(year=year, dest=site, monsoon)]
Unassigned$num_sampled[is.na(Unassigned$num_sampled)] <- 0
#sum(Unassigned$num_sampled, na.rm=T)==check1-37 #total should be the total sampled particles minus the total assigned
setorder(Unassigned, year, dest)

##adding in the possible sampled routes needs to happen AFTER calculating unassigned because unassigned is calculated from row counts
SimSample <- PropSampTable[SimSample, on=.(year, source, dest, monsoon)]
#check all is well- for testing loop only
#sum(SimSample$parentage) #should be 37

#add in the routes we could have assigned given our sampling so the parentage matrix is complete
UnqSimSample <- unique(SimSample, by=c("source", "dest", "year", "monsoon"))

AddRoutes <- UnqSurvey[!UnqSimSample, on = names(UnqSurvey)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , `:=`(parentage= 0, num_sampled = 0) ] #add the parentage column 

#add back into the sampled simulation data
l <- list(SimSample, AddRoutes[,-"num_sampled"])
SimSample <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = NULL)
setorder(SimSample, year, source, dest)

#make summary tables for each time frame, to be used for making parentage matrix
SimSampleByYear <- SimSample[,  .(total_parentage =sum(parentage)), by=c("year", "source", "dest")]
#sum(SimSampleByYear$total_parentage)
UnassignedByYear <- Unassigned[, .(total_sampled = sum(num_sampled)), by=c("year", "dest")]


SimSampleInterannual <- SimSample[,  .(total_parentage =sum(parentage)), by=c("source", "dest")]
#sum(SimSampleInterannual$total_parentage)
UnassignedInterannual <- Unassigned[, .(total_sampled = sum(num_sampled)), by=c("dest")]


SimSampleMonsoon <- SimSample[,  .(total_parentage =sum(parentage)), by=c("monsoon", "source", "dest")]
sum(SimSampleMonsoon$total_parentage)
UnassignedMonsoon <- Unassigned[, .(total_sampled = sum(num_sampled)), by=c("monsoon", "dest")]

#make a parentage matrix for each year
mat2012 <- dcast(SimSampleByYear[year==2012], source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year==2012 & prop_anem_samp >0, site]][, -"source"] 
mat2012 <- as.matrix(rbind(mat2012, t(UnassignedByYear[year==2012][, total_sampled]), use.names=F))

mat2013 <- dcast(SimSampleByYear[year==2013], source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year==2013 & prop_anem_samp >0, site]][, -"source"] 
mat2013 <- as.matrix(rbind(mat2013, t(UnassignedByYear[year==2013][, total_sampled]), use.names=F))

mat2014 <- dcast(SimSampleByYear[year==2014], source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year==2014 & prop_anem_samp >0, site]][, -"source"] 
mat2014 <- as.matrix(rbind(mat2014, t(UnassignedByYear[year==2014][, total_sampled]), use.names=F))

mat2012_4 <- dcast(SimSampleInterannual, source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0, site]][, -"source"] 
mat2012_4 <- as.matrix(rbind(mat2012_4, t(UnassignedInterannual[, total_sampled]), use.names=F))

matNEM <- dcast(SimSampleMonsoon[monsoon=="NEM"], source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0, site]][, -"source"] 
matNEM <- as.matrix(rbind(matNEM, t(UnassignedMonsoon[monsoon=="NEM"][, total_sampled]), use.names=F))

matSWM <- dcast(SimSampleMonsoon[monsoon=="SWM"], source ~ dest, value.var="total_parentage", fun.aggregate = sum)[source %in% SurveyData[year %in% c(2012, 2013, 2014) & prop_anem_samp >0, site]][, -"source"] 
matSWM <- as.matrix(rbind(matSWM, t(UnassignedMonsoon[monsoon=="SWM"][, total_sampled]), use.names=F))



In [ ]:
matSWM

In [ ]:
#Biophysical source normalized matrix
#for each source, what is the normalized recruitment at each destination? 
GenSimConn[, annual_source_normalized_recruitment := sum(daily_particles_recruited)/sum(daily_particles_released), by=c("source", "destination","year_sampled")]


__Code function for likelihood of parentage data given a biophysical model, based on Bode et al. 2019 in Plos Bio__

In [ ]:
#lay out all the pieces
pop_size_vec <- #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
BioPhysMat <- #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
prop_samp_vec <- #vector of proportion of habitat sampled for all reefs in time period t
unassigned_vec#from genetic parentage data- a vector of the number of unassigned recruits at each destination reef in the system- we only have this for all sampled reefs.... what should the dimensions be?*


In [ ]:
#format the data

In [ ]:
reef

In [ ]:
head(AnnualRecsSamp)

In [ ]:
#Eqn. S3.4 term 1 for loop

for(j in 1:nrow(centroids)){ #for each destination reef in the whole Camotes system
    
dest_unassigned <- sum(unassigned_vec[t==time_period]) #the sum of all unassigned recruits at time period t
    
        for(i in 1:nrow(centroids)){ #term 1- for each source reef in the whole Camotes system, expected unassigned at each destination reef
            
            term1_num <- ((1-prop_samp_vec[i,t== time_period])^2)*pop_size_vec[i]*BioPhysMat[i,j,t]
            
            for(a in 1:nrow(centroids)){ #for all reefs total in the whole Camotes system, denominator of term 1
                
                value_a <-  pop_size_vec[a]*BioPhysMat[a, j, t]
                term1_denom <- sum(value_a)
            }
        
        term1 = term1_num/term1_denom
        }

    for(k in 1:nrow(centroids)){ #term 2- for each source reef in the whole Camotes system, expected assigned at each destination reef
        
        term2_num <- (1-(1-prop_samp_vec[i,t== time_period])^2)*pop_size_vec[i]*BioPhysMat[i,j,t]
            
            for(a in 1:nrow(centroids)){ #for all reefs total in the whole Camotes system, denominator of term 2
                
                value_a <-  pop_size_vec[a]*BioPhysMat[a, j, t]
                term2_denom <- sum(value_a)
            }
        
        term2 = term2_num/term2_denom
        
        }
        
    

prob_unsampled <- dest_unassigned*log(term1)  
prob_sampled <- dest_assigned*log(term2)
ll = prob_unsampled + prob_sampled
}

In [ ]:

for(i in 1:NumSampledReefs){
   This_SS_A = Adult_sample_proportions[i]#same
   for(j in 1:NumSampledReefs){
    SettlersFromAssignedReefs = Settlers[Sampled_reefs[i],Sampled_reefs[j]]#same
    #Not all settlers from assigned reefs will be assigned, because not all adults were sampled
    AssignedSettlers[i,j] = SettlersFromAssignedReefs*(This_SS_A^2 + 2*This_SS_A*(1 - This_SS_A))
    AssignedSettlers[NumSampledReefs+1,j] = AssignedSettlers[NumSampledReefs+1,j] + SettlersFromAssignedReefs*(1-This_SS_A)^2 #The three dots '...' tell matlab that the code on a given line continues on the next line.
   }
}
Unsampled = as.matrix(setdiff(1:NumReefs,Sampled_reefs))


for(j in 1:NumSampledReefs){
   AssignedSettlers[NumSampledReefs+1,j] = AssignedSettlers[NumSampledReefs+1,j] + sum(Settlers[Unsampled,Sampled_reefs[,j]]) 
}
   



__Loop through sampling different proportions of other and CAI source particles and compare the unassigned proportions of total sample particles to the genetic observations from survey data__

In [ ]:
PropAssignedTable <- rbind(kernels[Year %in% c("2012", "2013", "2014")][
    , PropAssigned := PercentAssigned/100][ #change to proportion note percent
    , c("Year", "NumParentageMatches", "NumOffsSampled", "PropAssigned")],                      
    unique(kernels[Year %in% c("2012", "2013", "2014")][ #only the years coinciding with the models
    , `:=` (NumParentageMatches=sum(NumParentageMatches), NumOffsSampled=sum(NumOffsSampled), PropAssigned = NumParentageMatches/NumOffsSampled, Year = "2012-4")][ #summarise across the 3 years
    , c("Year","NumParentageMatches", "NumOffsSampled", "PropAssigned")], by="Year"))
    

PropAssignedTable[]

#add in the average sampled proportion of anemones
AvgPropSamp <- SurveyData[PropAnemSamp >0, .(PropAnemSamp = mean(PropAnemSamp)), by="year"][ #average for each site we sampled, how well we sampled
    year %in% c("2012", "2013", "2014")][
    , year :=as.character(year)]

ExpectedPropAssigned <- AvgPropSamp[PropAssignedTable, on=.(year=Year)]
ExpectedPropAssigned$PropAnemSamp[is.na(ExpectedPropAssigned$PropAnemSamp)] <- mean(ExpectedPropAssigned$PropAnemSamp, na.rm = T) #replace the 2012-4 NA with the average from the 3 years

#what's the normalized self recruitment proportion back to the population
ExpectedPropAssigned[, ExpAssigned := NumParentageMatches/(NumOffsSampled*PropAnemSamp)][] #this is the expected assignment for the whole surveyed population if we had sampled all adults (which we kind of do when we use the simulation results)
#at some point... maybe it would be better to compare on a site to site level? idk that's pretty fine scale, I don't know that our ROMS model can be expected to compare so well with that

__For the expected values of recruits from outside of our sampled region, I'm using the intermediate "PredictedProportions" matrix from the Bode kernel fitting script (https://github.com/MikeBode/Parentage_kernel_fitting/blob/master/Kernel_Fitting_Function.m) because it accounts for how well sites were sampled when estimating proportions of recruits from unsampled sites__

In [ ]:
#####outside of the loop*****

PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")]

#make sure all sampled sites are represented by joining the survey data to the sampled simulation
PropSampTable <- rbind(SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "NEM"], SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "SWM"])
#unq_survey <- unique(PropSampTable, by=c("Source", "Dest", "Year", "Monsoon"))#, unique(PropSampTable, by=c("Source", "Dest", "Year"))[, Monsoon := "SWM"]) #add in the diff Monsoon seasons so there are complete parentage matrices later
#add_dest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "NEM"], SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table

###outside of the loop
PropToEval <- seq(0.1, 1, 0.1) #make a vector of proportions to sample iterativaley and compare
#empty table to hold results
PropSampOtherCAI <- data.table(TimeScale=character(), TimeID=character(), PropUnassigned=numeric(), ExpUnassigned=numeric(),  PropSampEval=numeric(), Check1=character(), Check2=character(), NrowSimConn=numeric())



In [ ]:
pb <- txtProgressBar(min = 0, max =length(PropToEval), style = 3)

StartTime <- Sys.time()

for(i in 1:length(PropToEval)){

PropSampOtherCAI_int <- data.table(TimeScale=character(), TimeID=character(), PropUnassigned=numeric(), ExpUnassigned=numeric(),  PropSampEval=numeric(), Check1=character(), Check2=character(), NrowSimConn=numeric())[1:4]

dest_sampled <- date_join[DestPropSamp >0]
check1 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "Other"]], sample(dest_sampled[, .I[Source == "Other"]], length(dest_sampled[, .I[Source == "Other"]])*PropToEval[i]))]
check2 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "CAI"]], sample(dest_sampled[, .I[Source == "CAI"]], length(dest_sampled[, .I[Source == "CAI"]])*PropToEval[i]))]
check3 <- nrow(dest_sampled)

#check that we have less rows, should both be TRUE
test1 <- check1 > check2
test2 <- check2 > check3
#check1 > check2
#check2 > check3


#join in the number of parentage matches observed by year
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[

#randomly subsample the sampled particle data
sim_sample <- dest_sampled[, .SD[sample(.N, DestNOffsAnnual, prob=SurvWeight)], by = c("Year", "Dest")] #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys

PropUnassignedByYear <- (sim_sample[Source == "CAI"| Source == "Other", .(.N), by="Year"][, N]/#total particales sampled from other/CAI sources
sim_sample[, .(.N), by="Year"][, N]) #total particles sampled

PropSampOtherCAI_int$TimeScale[1] <- "annual"
PropSampOtherCAI_int$TimeID[1] <- "2012"
PropSampOtherCAI_int$PropUnassigned[1] <- PropUnassignedByYear[1]
PropSampOtherCAI_int$ExpUnassigned[1] <- 1-(ExpectedPropAssigned[1, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[1] <- PropToEval[i]
PropSampOtherCAI_int$Check1[1] <- test1
PropSampOtherCAI_int$Check2[1] <- test2
PropSampOtherCAI_int$NrowSimConn[1] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[2] <- "annual"
PropSampOtherCAI_int$TimeID[2] <- "2013"
PropSampOtherCAI_int$PropUnassigned[2] <- PropUnassignedByYear[2]
PropSampOtherCAI_int$ExpUnassigned[2] <- 1-(ExpectedPropAssigned[2, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[2] <- PropToEval[i]
PropSampOtherCAI_int$Check1[2] <- test1
PropSampOtherCAI_int$Check2[2] <- test2
PropSampOtherCAI_int$NrowSimConn[2] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[3] <- "annual"
PropSampOtherCAI_int$TimeID[3] <- "2014"
PropSampOtherCAI_int$PropUnassigned[3] <- PropUnassignedByYear[3]
PropSampOtherCAI_int$ExpUnassigned[3] <- 1-(ExpectedPropAssigned[3, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[3] <- PropToEval[i]
PropSampOtherCAI_int$Check1[3] <- test1
PropSampOtherCAI_int$Check2[3] <- test2
PropSampOtherCAI_int$NrowSimConn[3] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[4] <- "interannual"
PropSampOtherCAI_int$TimeID[4] <- "2012_4"
PropSampOtherCAI_int$PropUnassigned[4] <- nrow(sim_sample[Source == "CAI"| Source == "Other"])/nrow(sim_sample)
PropSampOtherCAI_int$ExpUnassigned[4] <- 1-(ExpectedPropAssigned[4, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[4] <- PropToEval[i]
PropSampOtherCAI_int$Check1[4] <- test1
PropSampOtherCAI_int$Check2[4] <- test2
PropSampOtherCAI_int$NrowSimConn[4] <- nrow(dest_sampled)

l <- list(PropSampOtherCAI, PropSampOtherCAI_int)
PropSampOtherCAI <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = FALSE)
setTxtProgressBar(pb, i)
    
}

close(pb)
EndTime <- Sys.time()
EndTime-StartTime

fwrite(PropSampOtherCAI, file="~/oceanography/script_output/SimulationSummaryTables/PropSampOtherCAIEvaluation.csv")


In [ ]:
PropSampOtherCAI[order(-PropUnassigned)]

__Seems like good justification to not subsample the ROMS particles from other/Camotes Islands__

In [ ]:
nrow(dest_sampled)
nrow(SimConn)

In [ ]:
#save inter file
#fwrite(dest_sampled, file="~/oceanography/script_output/LongFormConnWithProbsTest.csv")
#see if I can write as a compressed file so it can be stored on github
#https://stackoverflow.com/questions/42788401/is-possible-to-use-fwrite-from-data-table-with-gzfile